In [ ]:
# SLOW RETRIEVAL OF PLAYERS FROM API

# # add players to db
# #TODO: query all players from db at once instead of one at a time
# from nba_api.stats.endpoints import commonallplayers
# from nba_api.stats.endpoints import commonplayerinfo
# from nba_api.stats.endpoints import playercareerstats
# from nba_api.stats.library.parameters import PerModeSimple

# all_players = commonallplayers.CommonAllPlayers()
# data = all_players.get_normalized_dict()
# count = 0
# for player in data['CommonAllPlayers']:
#     if player["ROSTERSTATUS"] == 1:
#         names = player["DISPLAY_FIRST_LAST"].split(" ", 1)
#         player_data = {
#             "playerID":player["PERSON_ID"],
#             "name":player["DISPLAY_FIRST_LAST"],
#             "first_name":names[0],
#             "last_name":names[1],
#             "teamABR": player["TEAM_ABBREVIATION"],
#             "league":"NBA",
#             "seasons":[],
#         }
#         player_career = playercareerstats.PlayerCareerStats(player["PERSON_ID"], per_mode36=PerModeSimple.per_game)
#         regular_seasons = player_career.get_normalized_dict()["SeasonTotalsRegularSeason"]
#         time.sleep(2.5)
#         print(regular_seasons)
#         for season in regular_seasons:
#             # if season["MIN"] > 8.0:
#             player_data["seasons"].append(season["SEASON_ID"])
#             if season["SEASON_ID"] == "2022-23":
#                 print("2022-23")
#         if len(player_data["seasons"]) > 0:
#             info = commonplayerinfo.CommonPlayerInfo(player["PERSON_ID"]).get_normalized_dict()["CommonPlayerInfo"][0]
#             position=info["POSITION"]
#             # convert Center-Forward to C-F
#             foundDash=position.find("-")
#             if foundDash != -1:
#                 position=position[0]+"-"+position[position.index("-")+1]
#             else:
#                 position=position[0]
#             player_data["position"]=position
#             player_data["height"]=info["HEIGHT"]
#             player_data["weight"]=int(info["WEIGHT"])
#             player_data["lastModifiedDate"] = lastModifiedDate
#             res = db.players.update_one({"playerID": season["PLAYER_ID"]}, {"$set":player_data}, True)
#             count+=1
#             print(str(count) + ": " + player["DISPLAY_FIRST_LAST"] + " " + position)

In [4]:
# last retrieved OCT 19, 2:38AM EST
import json
playerindex = {}
with open("./playerindex_2022_23.json", "r") as f:
    playerindex = json.load(f)

In [5]:
headers=playerindex["0"]["headers"]
for i, h in enumerate(headers):
    print(i, h)


0 PERSON_ID
1 PLAYER_LAST_NAME
2 PLAYER_FIRST_NAME
3 PLAYER_SLUG
4 TEAM_ID
5 TEAM_SLUG
6 IS_DEFUNCT
7 TEAM_CITY
8 TEAM_NAME
9 TEAM_ABBREVIATION
10 JERSEY_NUMBER
11 POSITION
12 HEIGHT
13 WEIGHT
14 COLLEGE
15 COUNTRY
16 DRAFT_YEAR
17 DRAFT_ROUND
18 DRAFT_NUMBER
19 ROSTER_STATUS
20 FROM_YEAR
21 TO_YEAR
22 PTS
23 REB
24 AST
25 STATS_TIMEFRAME


In [15]:
import mysql.connector

db = mysql.connector.connect(
    host="us-east.connect.psdb.cloud",
    username="",
    password="",
    database="nba",
  )

In [24]:
import datetime
lastModifiedDate = datetime.datetime.now()
players = []
for player in playerindex["0"]["rowSet"]:
    if int(player[21]) < 2022:
        continue
    # start = int(player[20])
    # end = int(player[21])
    # years = [str(x) for x in range(start, end+1)]
    # # convert years 2020,2021,2022 to 2020-21, 2021-22, 2022-23
    # seasons = [str(years[i]) + "-" + str(int(str(years[i])[2:])+1) for i in range(len(years))]
    # convert height to inches
    ftInches = player[12].split("-")
    height = int(ftInches[0])*12 + int(ftInches[1])
    player_data = {
        "playerID":player[0],
        "name":player[2] + " " + player[1],
        "firstName":player[2],
        "lastName":player[1],
        "teamABR": player[9],
        "teamID": player[4],
        "position":player[11],
        "height":player[12],
        "heightInches":height,
        "weight":int(player[13]),
        "league":"NBA",
    }
    players.append(player_data)
print(len(players))

500


In [20]:
cursor = db.cursor()
query = "INSERT INTO players (playerID, name, firstName, lastName, teamABR, teamID, position, height, heightInches, weight, league) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
batch = []
for player in players:
    batch.append((player["playerID"], player["name"], player["firstName"], player["lastName"], player["teamABR"], player["teamID"], player["position"], player["height"], player["heightInches"], player["weight"], player["league"]))
    if len(batch) >= 20:
        cursor.executemany(query, batch)
        db.commit()
        batch = []